# Combinando DataSets. Merge y Join
pg 146

In [2]:
import pandas as pd
import numpy as np

### Categorias de joins

#### one to one

Muy silimar a concatenar columnas.

In [4]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [5]:
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [6]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


`pd.merge()` reconoce que cada DF tiene una columna llamada employee y automaticamente hace el join. El resultado es un nuevo DF que combina la información. El orden no se mantiene.

#### Muchos a uno

Una de las dos columnas clave contine duplicados. El DF resultante preserva esos duplicados.

In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [8]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


#### Muchos a muchos

Ambas contienen duplicados.

In [9]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
df5

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [10]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


*Similar a lo que me pasaba cuando creaba los nodos*

### Especificar la clave del Merge

#### on keyword

In [11]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Solo si ambos DF tienen esa columna

#### left_on, right_on

Para nombres de columnas diferentes.

In [12]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [13]:
pd.merge(df1, df3, left_on="employee", right_on="name")

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


El resultado tendrá una columna redundante que podemos quitar.

In [14]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### left_index, right_index

Mergear en los indices

In [15]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [16]:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [17]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


Los DF tienen un método `join()` que hace el merge directamente en los indices.

In [18]:
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


### Especificando la aritmética de los joins

Por defecto el resultado contiene la intersección de los dos sets. `inner join`.
Podemos modificarlo con la palabra `how`

In [19]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [20]:
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [21]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


`Outer` realiza la union y rellena con NaN.  
`Left y Right` Mantienen los del set especificado.

In [22]:
pd.merge(df6, df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [23]:
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [24]:
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


### Sobrescribiendo nombres de columnas

Si ambos datasets tienen columnas con igual nombre la funcion merge automaticamente pone un sufijo para mantener un nombre único. Podemos especificar el sufijo.

In [25]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})

In [26]:
pd.merge(df8, df9, on="name")

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [27]:
pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


### Ejemplo de uso

In [38]:
# Descargar los datos

In [29]:
!wget -P data/ https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!wget -P data/ https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!wget -P data/ https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

--2020-12-31 12:23:49--  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.132.133
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[151.101.132.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 57935 (57K) [text/plain]
Guardando como: “data/state-population.csv.1”

state-population.cs 100%[===================>]  56,58K  --.-KB/s    en 0,004s  

2020-12-31 12:23:49 (13,6 MB/s) - “data/state-population.csv.1” guardado [57935/57935]

--2020-12-31 12:23:49--  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.132.133
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[151.101.132.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 835 [text/plain]
Guardando como: “data/sta

In [39]:
# Visualizar los csv

In [31]:
pop = pd.read_csv('data/state-population.csv')

In [33]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [34]:
areas = pd.read_csv('data/state-areas.csv')

In [35]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [36]:
abbrevs = pd.read_csv('data/state-abbrevs.csv')

In [37]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [44]:
# Queremos calcular la densidad de población del 2010 y ordenar.

In [52]:
merged = pd.merge(pop, abbrevs, left_on='state/region', right_on='abbreviation', how='outer').drop('abbreviation', axis=1)

In [53]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [54]:
# mirar si hay nulos

In [55]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [59]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


In [60]:
# Puerto Rico no tiene datos de antes del 2000

In [71]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [72]:
# Localiza donde state es nulo y coge la columna state/region. Saca los valores unicos de la SERIE

In [73]:
# PR y USA no aparecen en el DF de abreviaciones. Vamos a solucionarlo.

In [77]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [78]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [79]:
# Ahora vamos a incluir el area.

In [80]:
final = pd.merge(merged, areas, on='state', how='left')

In [81]:
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [82]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [83]:
# Tenemos nulos en el area, vamos a revisar.

In [85]:
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

In [86]:
# El DF final no tiene el area de United States. Como no usaremos ni queremos esos datos vamos a descartarlos.

In [87]:
final.dropna(inplace=True)

In [88]:
data2010 = final.query("year == 2010 & ages == 'total'")

In [90]:
data2010.set_index('state', inplace=True)

In [91]:
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


In [92]:
density = data2010['population'] / data2010['area (sq. mi)']

In [93]:
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [95]:
density.sort_values(ascending=False, inplace=True)

In [96]:
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [97]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64